Il Chicago SRW (Soft Red Winter) Wheat è un tipo di grano invernale. Quindi, il prezzo che stai considerando è relativo al grano invernale. Il grano invernale viene seminato in autunno, in genere tra settembre e novembre, e viene raccolto l'anno successivo, generalmente tra maggio e luglio

In [8]:
#pip install yfinance lxml

Importiamo le librerie necessarie:

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
%matplotlib inline
import yfinance

Creiamo un oggetto `Ticker` che fornisce il metodo .history(), che chiameremo per ottenere un frame con i dati storici in un determinato periodo. Il metodo `history` prende in argomento 3 variabili:
- `start`, data della prima istanza del dataset.
- `end`, data dell'ultima istanza del dataset.
- `actions`, per ottenere delle colonne aggiuntive, che però noi setteremo a `False`.

In [9]:
chicagoSRW = yfinance.Ticker("ZW=F")
data = chicagoSRW.history(start = "2000-07-17", end = "2024-06-07", actions = False)
data.head()

,Open,High,Low,Close,Volume
Date,,,,,
2000-07-17 00:00:00-04:00,248.00,248.0,243.75,244.00,7233
2000-07-18 00:00:00-04:00,244.25,247.5,241.00,241.25,6523
2000-07-19 00:00:00-04:00,241.25,246.0,239.00,245.00,7136
2000-07-20 00:00:00-04:00,244.00,247.5,244.00,247.00,3792
2000-07-21 00:00:00-04:00,247.00,248.5,246.50,247.25,4380


Stampando le prime righe del frame, possiamo notare gli attributi di ogni istanza:
- `Date`, che è l'indice della riga.
- `Open`, prezzo di apertura giornaliero.
- `High`, prezzo massimo giornaliero.
- `Low`, prezzo minimo giornaliero.
- `Close`, prezzo di chiusura giornaliero.
- `Volume`, numero di titoli scambiati in giornata.